In [1]:
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.amazon.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.amazon.com/')

True

In [10]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [11]:
driver = webdriver.Chrome(executable_path=r'chromedriver.exe')
driver.maximize_window()

In [13]:
driver.get('https://tiki.vn')
time.sleep(1.)
driver.get('https://amazon.com')

In [14]:
elem = driver.find_element_by_id('twotabsearchtextbox')
elem.clear()
elem.send_keys("laptop")
elem.send_keys(Keys.RETURN)

In [ ]:
features = ['Screen Size', 'Processor', 'RAM', 'Hard Drive', 'Chipset Brand', 'Card Description', 'Number of USB 3.0 Ports', 'Brand Name', 
           'Series', 'Operating System', 'Item Weight', 'Product Dimensions', 'Item Dimensions L x W x H', 'Processor Brand',
           'Processor Count','Flash Memory Size','Hard Drive Interface', 'Hard Drive Rotational Speed', 'Optical Drive Type','Batteries']
features_with_price = features.copy()
features_with_price.append('Price')

In [ ]:
def extractData(text):
    res = ['']*len(features)
    for lines in text:
        liness = lines.text
        for line in liness.split('\n'):
            for i in range (len(features)):
                if features[i] in line and res[i] =='':
                    res[i] = line[len(features[i])+1:]
    return res

In [ ]:
file = open ('laptop.csv', 'w', encoding = 'utf-8')
output = csv.writer(file, delimiter = '\t')
output.writerow(features_with_price)
for url in url_list:
    time.sleep(.1)
    driver.get(url)
    print (url)
    infor = driver.find_elements_by_class_name('prodDetTable')
    data = extractData(infor)
    price = driver.find_elements_by_id('priceblock_ourprice')
    if price == []:
        price = browser.find_elements_by_id('priceblock_dealprice')
    if price != []:
        price = price[0].text
    else:
        price = ''
    data.append(price)
    output.writerow(data)
file.close()